# Dealing with Imbalanced Class Variable

In classification problems, sometimes we may encounter datasets that have class imbalance issue: a class value dominates the other class values. This poses significant challenges to the classification algorithms which work well with balanced class. This type of class imbalance problem can be found in areas like medical diagnosis, spam filtering, and fraud detection.

In this lab, we'll look at possible ways to handle an imbalanced class problem using a credit card data. Here each transaction is tagged as fradulent (1) or not fradulent (0). Our objective will be to correctly classify the minority class of fraudulent transactions.

Important Note: This guide will focus soley on addressing imbalanced classes and will not addressing other important machine learning steps including, but not limited to, feature selection or hyperparameter tuning.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, classification_report
from sklearn.metrics import fbeta_score

%matplotlib inline

In [ ]:
# read in data
datapath = "./credit_card.csv"
df = pd.read_csv(datapath)

print(df.shape)

# show random 5 rows from this dataset
df.sample(n=5, replace=False)

In [ ]:
# print the class 
df.Class.value_counts()

In [ ]:
# print percentage of questions where target == 1
fraud_ratio = df[df.Class==1].shape[0] * 100 / df.shape[0]
fraud_ratio

We can see we have a very imbalanced class - just 0.17% of our dataset belong to fradulent transactions!

This is a problem because many machine learning models are designed to maximize overall accuracy, which especially with imbalanced classes may not be the best metric to use. Classification accuracy is defined as the number of correct predictions divided by total predictions times 100. For example, if we simply predicted all transactions are not fraud, we would get a classification acuracy score of over 99%!

# Create Train and Test Sets
The training set is used to build and validate the model, while the test set is reserved for testing the model on unseen data.

In [ ]:
# Prepare data for modeling
# Separate input features and target
y = df.Class
X = df.drop('Class', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

In [ ]:
y_train.value_counts()[1] * 100 / y_train.shape[0]

In [ ]:
y_test.value_counts()[1] * 100 / y_test.shape[0]

# Baseline Models

In [ ]:
# DummyClassifier to predict only target 0
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)

# checking unique labels
print('The predicted labels: ', np.unique(dummy_pred))

# checking accuracy
print('Accuracy score: ', accuracy_score(y_test, dummy_pred))

print(classification_report(y_test, dummy_pred))

The accuracy score for this Dummy Classifier is 99.8%, and this highy accuracy may mislead us. As the Dummy Classifier predicts only Class 0, it is clearly not a good option for our objective, which is correctly classifying fraudulent transactions.

Let's see how logistic regression performs on this dataset.

In [ ]:
# Modeling the data as is
# Train model
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# Predict on training set
lr_pred = lr.predict(X_test)

In [ ]:
# print confusion matrix
pd.DataFrame(confusion_matrix(y_test, lr_pred))

In [ ]:
print(classification_report(y_test, lr_pred))

In [ ]:
# Checking accuracy to see the first few digits after decimal point
accuracy_score(y_test, lr_pred)

In [ ]:
# Checking unique values
predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

Logistic Regression outperformed the Dummy Classifier. We can see that it predicted 4 instances of class 1, so this is definitely an improvement. But can we do better?

Let's see if we can apply some techniques for dealing with class imbalance to improve these results. 

### 1. Change the evaluation measure

We should not use accuracy as a default measure for evaluating an imbalanced class. Metrics that can provide better insight include:
 
* **Confusion Matrix:** a table showing the number of correct and incorrect predictions
* **Precision:** the number of true positives divided by all positive predictions. Precision is also called Positive Predictive Value. It is a measure of a classifier's exactness. Low precision indicates a high number of false positives.
* **Recall:** the number of true positives divided by the number of positive values in the test data. Recall is also called Sensitivity or the True Positive Rate. It is a measure of a classifier's completeness. Low recall indicates a high number of false negatives.
* **F1-Score:** the weighted average of precision and recall
* **F-beta Score:** It's a generailzation of F1-score. F1-score give equal weights to precision and recall. In some case precision could be more important than recall and vice versa. Check this short [article](https://onlinehelp.explorance.com/blueml/Content/articles/getstarted/mlcalculations.htm?TocPath=Get%20started%7C_____3) on F-beta measure. 

If our main objective is to classifying the fraud cases the recall score (or F-2 score) can be considered a key metric to use for evaluating predictions.

In [ ]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, lr_pred))

In [ ]:
# recall score
recall_score(y_test, lr_pred)

In [ ]:
# f1 score
f1_score(y_test, lr_pred)

In [ ]:
# F2 Score
fbeta_score(y_test, lr_pred, beta=2)

We have a very high accuracy score of 0.999 but a F1 score of only 0.752. And from the confusion matrix, we can see we are misclassifying several observations leading to a recall score of only 0.64.

### 2. Change the learner

For every machine learning problem, its a good rule of thumb to try a variety of algorithms, it can be beneficial with imbalanced datasets. Decision trees frequently perform well on imbalanced data. They work by learning a hierachy of if/else questions. This can force both classes to be addressed.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# train model
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

print(classification_report(y_test, rfc_pred))

In [ ]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, rfc_pred))

It seems RandomForest was able to overcome the class imbalance problem for this dataset. The results on the test set show higher recall and F1-score compared to logistic regression. 

### 3. Resampling Techniques

#### Oversampling Minority Class
Oversampling can be defined as adding more copies of the minority class. Oversampling can be a good choice when you don't have a ton of data to work with. A con to consider when undersampling is that it can cause overfitting and poor generalization to your test set.

We will use the resampling module from Scikit-Learn to randomly replicate samples from the minority class.

Important Note
Always split into test and train sets BEFORE trying any resampling techniques! Oversampling before splitting the data can allow the exact same observations to be present in both the test and train sets! This can allow our model to simply memorize specific data points and cause overfitting.

In [ ]:
from sklearn.utils import resample

In [ ]:
# concatenate our training data back together before doing the resampling

train_set = pd.concat([X_train, y_train], axis=1)
train_set.head()

In [ ]:
# separate minority and majority classes
train_set_not_fraud = train_set[train_set.Class==0]
train_set_fraud = train_set[train_set.Class==1]

# upsample minority
fraud_upsampled = resample(train_set_fraud,
                          replace=True, # sample with replacement
                          n_samples=train_set_not_fraud.shape[0], # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
train_set_upsampled = pd.concat([train_set_not_fraud, fraud_upsampled])

# check new class counts
train_set_upsampled.Class.value_counts()

In [ ]:
# trying logistic regression again with the balanced dataset
X_train_upsampled = train_set_upsampled.drop('Class', axis=1)
y_train_upsampled = train_set_upsampled.Class


upsampled = LogisticRegression(solver='liblinear').fit(X_train_upsampled, y_train_upsampled)
upsampled_pred = upsampled.predict(X_test)

print(classification_report(y_test, upsampled_pred))

In [ ]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, upsampled_pred))

The accuracy score (0.97) dropped after upsampling, but the model is now predicting all the fraud instances  correctly, an improvement over our plain logistic regression above.



#### Undersampling Majority Class

Undersampling can be defined as removing some observations of the majority class. Undersampling can be a good choice when you have a lot of data - think millions of rows. But a drawback to undersampling is that we are removing information that may be valuable.

We will again use the resampling module from Scikit-Learn to randomly remove samples from the majority class.

In [ ]:
# still using our separated classes fraud and not_fraud from above

# downsample majority
not_fraud_downsampled = resample(train_set_not_fraud,
                                replace = False, # sample without replacement
                                n_samples = train_set_fraud.shape[0], # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
train_set_downsampled = pd.concat([not_fraud_downsampled, train_set_fraud])

# checking counts
train_set_downsampled.Class.value_counts()

In [ ]:
# trying logistic regression again with the undersampled dataset

y_train_downsampled = train_set_downsampled.Class
X_train_downsampled = train_set_downsampled.drop('Class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train_downsampled, y_train_downsampled)
undersampled_pred = undersampled.predict(X_test)

print(classification_report(y_test, undersampled_pred))

In [ ]:
# confusion matrix
pd.DataFrame(confusion_matrix(y_test, undersampled_pred))

The accuracy score (0.85) dropped after downsampling, but the model is predicting the minority class correctly although it performs worse compared to the upsampling technique.



### Generate Synthetic Samples

SMOTE or Synthetic Minority Oversampling Technique is a popular algorithm to create sythetic observations of the minority class.

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE()
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)
smote_pred = smote.predict(X_test)

In [ ]:
# confustion matrix
pd.DataFrame(confusion_matrix(y_test, smote_pred))

In [ ]:
print(classification_report(y_test, smote_pred))

SMOTE performs better than upsampling and downsampling techniques.  

# Save your notebook, then `File > Close and Halt`